# Autoformalism with LLMs

Formalism for mathematics is the process of translating a mathematical theorem or proof into fully formal logic.  In practice, the target of the translation is ussually a proof assistant language like Coq, Isabelle, or Lean, which encode formal logic in a manner that can be checked by a computer.  This notebook is a hands on tutorial for how to achieve this formalism automatically with Large Language Models (LLMs). 

We will closely follow the paper [Autoformalism with Large Language Models](https://arxiv.org/abs/2205.12615) (Wu et al.) with the execption that we will be using GPT-4 and Llama3 as our langauge model instead of Palm and Codex.


## Setup
You must put a file `.env` in the same directory as this notebook with the following contents:
```
OPENAI_API_KEY=<your_openai_api_key>
REPLICATE_API_TOKEN=<your_replicate_api_token>
```
replacing <your_...> with the actual keys provided by OpenAI and Replicate.  If you only want to run one of the APIs (e.g. only GPT-4 and not Llama3 on replicate) you can provide only the relevant key.

In [1]:
import re
from dataclasses import dataclass, asdict
import os
from dotenv import load_dotenv

# must come before importing openai and replicate
load_dotenv()

import openai 
import replicate 

from ml4math.datasets import MiniF2F, MiniF2FQuestion

----
## MiniF2F Dataset

We will look at the [MiniF2F dataset](https://github.com/facebookresearch/miniF2F/tree/main) which contains mathematical statements together with their formalized statements.  Since we have a pairing of informal and formal mathematics we can do two things:

1. We can use these pairing to create few-shot examples for translating form informal to formal mathematics.
2. We can use these pairing to evaluate how well the LLMs can translate informal to formal mathematics.

First let's load the dataset and look at an example to get a feel for the data.

In [2]:
minif2f = MiniF2F()

print(f"Number of statements in the dataset: {len(minif2f)}")

Number of statements in the dataset: 488


There are 488 statements from the dataset.  Let's see what a single example looks like.

In [3]:
minif2f[0]

MiniF2FQuestion(formal_statement='(*\n  Authors: Albert Qiaochu Jiang\n*)\n\ntheory mathd_algebra_11 imports\n  Complex_Main\nbegin\n\ntheorem mathd_algebra_11:\n  fixes a b :: real\n  assumes h0 : "a \\<noteq> b"\n    and h1 : "a \\<noteq> 2 * b"\n    and h2 : "(4*a+3*b) / (a-2*b) = 5"\n  shows "(a+11*b) / (a-b) = 2"\nproof -\n  have p0:"a-b \\<noteq> 0" using h0 by simp\n  have "a-2*b \\<noteq> 0" using h1 by simp\n  hence "(4*a+3*b) = 5 * (a-2*b)" using h2\n    by (metis Groups.mult_ac(2) nonzero_mult_div_cancel_left times_divide_eq_right)\n  also have "5 * (a-2*b) = 5 * a - 10 * b" by simp\n  ultimately have "4*a+3*b = 5 * a - 10 * b" by simp\n  hence h3: "a=13*b" by linarith\n  have "a + 11 * b = 2 * (a-b)" using p0 \n    unfolding h3 by auto\n  then show ?thesis using p0\n    by force\nqed\n\nend', informal_statement='What is the simplified numerical value of $\\frac{a+11b}{a-b}$ if $\\frac{4a+3b}{a-2b}=5$? Show that it is 2.', informal_proof="Let's play with the given condition 

An example is an dataclass of type `MiniF2FQuestion`.  Let's look at what types of fields are in this dataclass.

In [4]:
print("MiniF2F Question fields:")
for field in minif2f[0].__dataclass_fields__.keys():
    print(f" - {field}")

MiniF2F Question fields:
 - formal_statement
 - informal_statement
 - informal_proof
 - formal_proof
 - id


Let's checkout what the formal and informal statements looks like

In [5]:

print("Informal Statement: \n" + minif2f[1].informal_statement)
print("\n\n")
print("Formal Statement: \n" + minif2f[1].formal_statement)

Informal Statement: 
For any real number a, show that $10a \leq 28a^2 + 1$.



Formal Statement: 
(*
  Authors: Albert Qiaochu Jiang
*)

theory algebra_binomnegdiscrineq_10alt28asqp1 imports
  Complex_Main
begin

theorem algebra_binomnegdiscrineq_10alt28asqp1:
  fixes a :: real
  shows "10 * a \<le> 28 * a^2 + 1"
  sorry

end


We see the informal statement asks us to show that for any real number $a$, that $$10a \leq 28a^2 + 1$$  The formal statement is written in the proof assistant language of [Isabelle](https://isabelle.in.tum.de/).  It starts with a comment, which declares the author, next it declares an import to bring in some libraries for dealing with complex numbers, and finally it states the question as a theorem in Isabelle's language.

Note the use of `sorry` in the theorem, which is just a place holder indicating that we don't have a proof for this theorem yet.

-----
## Autoformalism 
**Goal:** Translate the informal statement into the formal statement.

Now that we've seen an example of a formal statement and the corresponding informal statement, we can understand our actual goal better.  We'd like an LLM to take the informal statement and automatically translate it to the corresponding formal statement, written in Isabelle code.  


**Approach**

1. Clean up the formal statements to remove imports and comments.
2. Generate a prompt for few-shot examples
3. Use an API to prompt an LLM to translate the informal statement to the formal statement.

#### Step 1: Clean up the formal statements

In [6]:
def remove_content_before_theorem(formal_statement: str) -> str:
    """Removes all the content before the theorem statement."""
    for line_number, line in enumerate(formal_statement.splitlines()):
        if re.search(r"^\s*theorem", line):
            return "\n".join(formal_statement.splitlines()[line_number:])
    return formal_statement

def remove_content_after_theorem_shows(formal_statement: str) -> str:
    """Remove the content after the shows statement in the theorem.

    Note:
        This is not applicable to metamath or hollight datasets.
    """
    for line_number, line in enumerate(formal_statement.splitlines()):
        if re.search(r"^\s*shows", line):
            return "\n".join(formal_statement.splitlines()[: line_number + 1])
    return formal_statement


def remove_theorem_name(formal_statement: str) -> str:
    """Removes the theorem name from the formal statement."""
    return re.sub(r"(.*theorem).*(?:|$)", r"\1", formal_statement, re.M)



def clean_formal_statement(formal_statement:str)->str:
    """Clean the formal statement.""" 
    formal_statement = remove_content_before_theorem(formal_statement)
    formal_statement = remove_content_after_theorem_shows(formal_statement)
    formal_statement = remove_theorem_name(formal_statement)
    return formal_statement

Let's check the cleaning process

In [7]:
example = minif2f[1]
print('-'*20 + "Original Formal Statement" + '-'*20)
print(example.formal_statement)
print('-'*20 + "Cleaned Formal Statement" + '-'*20)
print(clean_formal_statement(example.formal_statement))

--------------------Original Formal Statement--------------------
(*
  Authors: Albert Qiaochu Jiang
*)

theory algebra_binomnegdiscrineq_10alt28asqp1 imports
  Complex_Main
begin

theorem algebra_binomnegdiscrineq_10alt28asqp1:
  fixes a :: real
  shows "10 * a \<le> 28 * a^2 + 1"
  sorry

end
--------------------Cleaned Formal Statement--------------------
theorem
  fixes a :: real
  shows "10 * a \<le> 28 * a^2 + 1"


Technically every theorem in Isabelle needs a unique name, but we don't want the arbitary naming choices to effect our evaluation so we just strip the names of the theorems.  Let's move one to generating a few shot prompt.

### Step 2: Generate a prompt for few-shot examples

We want to stay consistent with the [Autoformalism with Large Language Models](https://arxiv.org/abs/2205.12615) paper so we need to use the same questions they used for the few-shot examples.  To do this, we searched the MiniF2F dataset manually to match find the questions used in the prompt, shown in the paper's appendix.  We compiled these question ids into a list and will use them to generate the few-shot examples.

One other consideration is that the MiniF2F dataset covers different topics, like algebra and number theory.  In the paper they use subject specific prompts, so we will do the same.  This also means we will need to filter the dataset to only include the questions from the specific subject we are interested in, for a given prompt.

In [8]:
@dataclass(frozen=True)
class FEWSHOTIDS:
    """IDS of the few-shot learning examples used in the paper

    We found these by copying snippets of the questions from the paper appendix and
    searching the dataset for matching questions.  See the accompying notebook
    `find_fewshot_ids.ipynb` for more details.
    """

    algebra: tuple[str, ...] = (
        "mathd_algebra_245",
        "mathd_algebra_76",
        "mathd_algebra_478",
        "mathd_algebra_338",
        "mathd_algebra_422",
        "mathd_algebra_43",
        "mathd_algebra_756",
        "mathd_algebra_149",
        "mathd_algebra_48",
        "mathd_algebra_410",
    )

    numbertheory: tuple[str, ...] = (
        "mathd_numbertheory_709",
        "mathd_numbertheory_461",
        "mathd_numbertheory_466",
        "mathd_numbertheory_257",
        "mathd_numbertheory_34",
        "mathd_numbertheory_780",
        "mathd_numbertheory_233",
        "mathd_numbertheory_764",
        "mathd_numbertheory_345",
        "mathd_numbertheory_227",
    )


##### Filter dataset to specific subject

We'll now filter to the algebra and number theory questions that come from the [MATH](https://github.com/hendrycks/math/) dataset.  This is the original set of questions used in the paper.

In [9]:
alg_full = [x for x in minif2f if 'mathd_algebra' in x.id]
num_full = [x for x in minif2f if 'mathd_numbertheory' in x.id]


##### Create Fewshot questions

In [10]:
def get_fewshot_examples(dataset, ids):
    examples = [x for x in dataset if x.id in ids]
    return sorted(examples, key=lambda x: ids.index(x.id))

def remove_fewshot_examples(dataset, ids):
    return [x for x in dataset if x.id not in ids]
    

alg_fewshot = get_fewshot_examples(alg_full, FEWSHOTIDS.algebra)
alg = remove_fewshot_examples(alg_full, FEWSHOTIDS.algebra)

num_fewshot = get_fewshot_examples(num_full, FEWSHOTIDS.numbertheory)
num = remove_fewshot_examples(num_full, FEWSHOTIDS.numbertheory)

assert len(alg_fewshot) == len(FEWSHOTIDS.algebra)
assert len(alg_full) == len(alg) + len(alg_fewshot)
assert len(num_fewshot) == len(FEWSHOTIDS.numbertheory)
assert len(num_full) == len(num) + len(num_fewshot)

##### Prompt Helpers

We make some helper functions to go from questions to fully formated prompts.

In [11]:
def make_fewshot_messages(fewshot_examples:list[MiniF2FQuestion])->list[dict]:
    """Takes a list of example questions and returns a list of messages for a chat style LLM.
    
    Args:
        fewshot_examples (list[MiniF2FQuestion]): Few-shot examples to be used.

    Returns:
        list[dict]: A list of chatbot style messages. 
    """
    messages = []
    for example in fewshot_examples:
        user_msg = {
            "role": "user",
            "content": example.informal_statement,
        }
        ai_msg = {
            "role": "assistant",
            "content": clean_formal_statement(example.formal_statement),
        }
        messages.append(user_msg)
        messages.append(ai_msg)
    return messages


def system_message()->dict:
    return {
        "role": "system",
        "content": ("Translate the following natural language math problem to the Isabelle theorem proving language.  "
                    "Do not provide a proof of the statement. Use dilligence when translating the problem and make "
                    "certain you capture all the necessary assumptions as hypotheses."),
    }

def prompt_messages(question:MiniF2FQuestion, few_shot_examples:list[MiniF2FQuestion])->list[dict]:
    """Returns a list of messages for the chat style LLM.
    
    Args:
        question (MiniF2FQuestion): The question to be answered.
        few_shot_examples (list[MiniF2FQuestion]): Few-shot examples to be used.

    Returns:
        list[dict]: A list chatbot style messages. 
    """
    messages = []
    messages.append(system_message())
    messages.extend(make_fewshot_messages(few_shot_examples))
    user_msg = {
        "role": "user",
        "content": question.informal_statement,
    }
    messages.append(user_msg)
    return messages


def convert_messages_to_llama3(messages: list[dict]) -> str:
    """Convert a list of messages to a llama3 string.

    See:
        https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/

    Args:
        messages (list[dict]): A list of messages.

    Returns:
        str: The llama3 string.
    """
    HEADER_START = "<|start_header_id|>"
    HEADER_END = "<|end_header_id|>"
    role_template = HEADER_START + "{role}" + HEADER_END + "\n\n"
    llama3 = []
    llama3.append("<|begin_of_text|>")
    for message in messages:
        msg = role_template.format(role=message["role"])
        msg += message["content"]
        msg += "<|eot_id|>"
        llama3.append(msg)

    llama3.append(role_template.format(role="assistant"))
    return "".join(llama3)


Let's check the prompt to make sure it looks good.

In [12]:
prompt_messages(alg[0], alg_fewshot)

[{'role': 'system',
  'content': 'Translate the following natural language math problem to the Isabelle theorem proving language.  Do not provide a proof of the statement. Use dilligence when translating the problem and make certain you capture all the necessary assumptions as hypotheses.'},
 {'role': 'user',
  'content': 'Simplify $\\left( \\frac{4}{x} \\right)^{-1} \\left( \\frac{3x^3}{x} \\right)^2 \\left( \\frac{1}{2x} \\right)^{-3}$. Show that it is 18x^8.'},
 {'role': 'assistant',
  'content': 'theorem\n  fixes x :: real\n  assumes h0 : "x \\<noteq> 0"\n  shows "1/(4/x) * ((3*x^3)/x)^2 * (1/(1 / (2 * x)))^3 = 18 * x^8"'},
 {'role': 'user',
  'content': 'For integers $n$, let \\[f(n) = \\left\\{\n\\begin{array}{cl}\nn^2 & \\text{ if }n\\text{ is odd}, \\\\\nn^2 - 4n - 1 & \\text{ if }n\\text{ is even}.\n\\end{array}\n\\right.\\]Find $f(f(f(f(f(4)))))$. Show that it is 1.'},
 {'role': 'assistant',
  'content': 'theorem\n  fixes f :: "int \\<Rightarrow> int"\n  assumes "\\<forall>n.

Notice the style of our few-shot prompt is in user/assistant pairs.  In the original paper the authors did not use a chat model and so put the few shot examples in a single user message. We will use the chat model style for our prompt.

### Step 3: Use GPT-4 or Llama3

We will now use GPT-4 or Llama3 to translate translate the informal statements to formal Isabelle code.  

**IMPORTANT!**

You must set the environment variables `OPENAI_API_KEY` and `REPLICATE_API_TOKEN` to use the OpenAI and Replicate APIs.  You can get an OpenAI API key by signing up at [OpenAI](https://platform.openai.com/).  You can get a Replicate API token by signing up at [Replicate](https://replicate.com/).  Set these by putting them in a `.env` file in the same directory as this notebook.  See the setup section in the introduction of this notebook for more details

In [13]:
@dataclass
class CONFIG:
    model: str 
    temperature: float = 0.0
    max_tokens: int = 512
    top_p: float = 1.0




def call_openai(messages, **kwargs):
    """Calls the OpenAI API with the given message and kwargs.""" 
    client = openai.Client()
    response = client.chat.completions.create(
        messages=messages,
        **kwargs
    )
    return response


def call_replicate(prompt, **kwargs):
    """Calls the Replicate API with the given prompt and kwargs."""
    model = kwargs.pop('model')
    kwargs.update(dict(
        prompt=prompt,
        prompt_template="{prompt}" 
    ))
    response = replicate.run(model, input=kwargs)
    return response



#### Call the API

We will now call the API to get the translation of the informal statement to the formal statement.  We are only going to execute 3 examples from the algebra dataset.  To see the full results and analysis on both the algebra and number theory dataset, see our [other repository](https://github.com/agencyenterprise/autoformalism-with-llms) where we reproduce to in-context learning results from the paper [Autoformalism with Large Language Models](https://arxiv.org/abs/2205.12615) by running the entire dataset through the APIs and analyzing the results.

In [14]:
GPT4_CONFIG = CONFIG(model='gpt-4-turbo-2024-04-09')
LLAMA3_CONFIG = CONFIG(model='meta/meta-llama-3-70b')

responses = []

for i in range(3):
    example = alg[i]
    messages = prompt_messages(example, alg_fewshot)
    llama_prompt = convert_messages_to_llama3(messages)

    try:
        gpt4 = call_openai(messages, **asdict(GPT4_CONFIG)).choices[0].message.content
    except Exception as e:
        print(f"Error calling OpenAI: {e}")
        gpt4 = "ERROR"
    
    try:
        llama3 = "".join(call_replicate(llama_prompt, **asdict(LLAMA3_CONFIG)))
    except Exception as e:
        print(f"Error calling Replicate: {e}")
        llama3 = "ERROR"

    responses.append({
        "gpt4": gpt4,
        "llama3": llama3,
    })

In [18]:
for i, response in enumerate(responses):
    print('-'*20 + f"Original Question, Alebra {i}" + '-'*20)
    print(alg[i].informal_statement)
    print('-'*20 + "Correct Reference Translation" + '-'*20)
    print(clean_formal_statement(example.formal_statement))

    response = responses[i]
    if response['gpt4'] != 'ERROR':
        print('-'*20 + "GPT-4 Translation" + '-'*20)
        print(response['gpt4'])
    if response['llama3'] != 'ERROR':
        print('-'*20 + "LLAMA-3 Translation" + '-'*20)
        print(response['llama3'])
    print('\n\n')

--------------------Original Question, Alebra 0--------------------
What is the simplified numerical value of $\frac{a+11b}{a-b}$ if $\frac{4a+3b}{a-2b}=5$? Show that it is 2.
--------------------Correct Reference Translation--------------------
theorem
  fixes a b :: real
    and f :: "real \<Rightarrow> real"
  assumes h0 : "\<And>x. f x = x^2 + a*x + b"
    and h1 : "2 * a \<noteq> b"
    and h2 : "f (2 * a) = 0"
    and h3 : "f b = 0"
  shows "a + b = -1"
--------------------GPT-4 Translation--------------------
theorem
  fixes a b :: real
  assumes "a \<noteq> 2 * b"
    and "(4 * a + 3 * b) / (a - 2 * b) = 5"
  shows "(a + 11 * b) / (a - b) = 2"
--------------------LLAMA-3 Translation--------------------
theorem
  fixes a b :: real
  assumes h0 : "(4 * a + 3 * b) / (a - 2 * b) = 5"
    and h1 : "a \<noteq> b"
  shows "(a + 11 * b) / (a - b) = 2" 



--------------------Original Question, Alebra 1--------------------
Let $f(x)=3x^4-7x^3+2x^2-bx+1$. For what value of $b$ is $f(1)=1

And we're done!  We've successfully used an LLM to translate informal mathematics to formal mathematics.  One of the challenging bottlenecks is there is no automatic way to evaluate the correctness of the translation.  If the code does not compile in Isabelle that's a fairly good sign that its not correct, but if it does compile then we know its *syntactically correct* but we don't know if its *semantically* correct.  Looking at the above examples, we can see that sometimes the LLM changes the hypothesis order, or names a function `p` for polynomial while the reference names it `f`.  These types of differences don't change the correctness but make the task very difficult to evaluate without human review. In the Autoformalism paper they manually review 50 examples from the Algebra and Number Theory dataset.  

Note that just because autoformalism is hard to evaluate - it does not make it useless.  In the papers [Autoformalism with Large Language Models](https://arxiv.org/abs/2205.12615) and [Draft, Sketch, Prove](https://arxiv.org/abs/2210.12283) the authors use autoformalism as a step in a larger process and show the entire pipeline can beat state of the art theorem proving (at the time).  

## Conclusion
In this notebook we showed how one can use the MiniF2F dataset to create few-shot examples for translating informal mathematics to formal mathematics.  We followed the spirit of the  [Autoformalism with Large Language Models](https://arxiv.org/abs/2205.12615) but showed how to prompt GPT-4 and Llama3 models using a API (as a paid service).  We hope this short tutorial peaks your interest in formal methods and how it can be possible to leverage deep learning to contribute to mathetmatics and formal methods.